In [16]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [17]:
program_id = PublicKey("4dSqc3aSSxM6UGNLQPc1LWMhoMniTSuzTdAryvcH53K8")
program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
program_id_account_meta

AccountMeta(pubkey=4dSqc3aSSxM6UGNLQPc1LWMhoMniTSuzTdAryvcH53K8, is_signer=False, is_writable=False)

In [18]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [19]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 600000000)

mint_account = Keypair.generate()

seed1 = bytes([225,215,211,54,213,18,25,122]*4)
vault_keypair = Keypair.from_seed(seed1)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z"), False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

# transaction = Transaction()
# transaction.add(
#   create_account(CreateAccountParams(from_pubkey=payer_account_meta.pubkey, new_account_pubkey=mint_account.public_key, lamports=Token.get_min_balance_rent_for_exempt_for_mint(client), space=MINT_LAYOUT.sizeof(), program_id=spl_program_meta.pubkey))
# )
# transaction.add(
#   initialize_mint(InitializeMintParams(
#     decimals=6, 
#     program_id=spl_program_meta.pubkey, 
#     mint=mint_account.public_key,
#     mint_authority=program_id, 
#     freeze_authority=program_id
#   ))
# )
   
# client.send_transaction(transaction, payer_keypair, mint_account)

In [20]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)
(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)


accounts = [
    # program_id_account_meta,
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    AccountMeta(mint_authority, False, False),
    AccountMeta(locktime_pda, False, True),
    AccountMeta(sales_pda, False, True),

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    ]

In [21]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([])
))
client.send_transaction(transaction, payer_keypair, mint_account)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: Failed to serialize or deserialize account data: Unknown', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'BorshIoError': 'Unknown'}]}, 'logs': ['Program 4dSqc3aSSxM6UGNLQPc1LWMhoMniTSuzTdAryvcH53K8 invoke [1]', 'Program 4dSqc3aSSxM6UGNLQPc1LWMhoMniTSuzTdAryvcH53K8 consumed 18435 of 200000 compute units', 'Program 4dSqc3aSSxM6UGNLQPc1LWMhoMniTSuzTdAryvcH53K8 failed: Failed to serialize or deserialize account data: Unknown']}}

In [ ]:
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

PDA: (53Q9ze9A87bzyDz4seTkN6UQMaKLNWeQTyh21PLwnomC, 253)


Container(vault_total=1.0, counter=1)

In [ ]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_data" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {locktime_pda, locktime_nonce}")

getAccountInfo(client,locktime_pda)

PDA: (BaeYeWbTyFyBSFHiJ1n2TqcWXu6EaDC4diXwoeZTdkks, 253)


TypeError: 'NoneType' object is not subscriptable

PDA: (53Q9ze9A87bzyDz4seTkN6UQMaKLNWeQTyh21PLwnomC, 253)


Container(vault_total=1.0, counter=1)